# Single Allocation P-hub Location Median Problem using Genetic Algorithm

## Importing Package and Dataset

In [1]:
# Importing Package and Libraries
import numpy as np
import pandas as pd
import random
import itertools
import time
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
# Importing Dataset Flow
# This dataset consists of flow from each nodes
CAB_10_nodes_flow = pd.read_csv("10_nodes_CAB_flow.csv", delimiter= ";", header = None)
CAB_25_nodes_flow = pd.read_csv("25_nodes_CAB_flow.csv", delimiter= ";", header = None)
TR_55_nodes_flow = pd.read_csv("55_nodes_TR_flow.csv", delimiter= ";", header = None)
TR_81_nodes_flow = pd.read_csv("81_nodes_TR_flow.csv", delimiter= ";", header = None)
RGP_100_nodes_flow = pd.read_csv("100_nodes_RGP_flow.csv", delimiter= ";", header = None)                                     

In [3]:
CAB_10_nodes_flow

,0,1,2,3,4,5,6,7,8,9
0,0,6469,7629,20036,4690,6194,11688,2243,8857,7248
1,6469,0,12999,13692,3322,5576,3878,3202,6699,4198
2,7629,12999,0,35135,5956,14121,5951,5768,16578,4242
3,20036,13692,35135,0,19094,35119,21423,27342,51341,15826
4,4690,3322,5956,19094,0,7284,3102,1562,7180,1917
5,6194,5576,14121,35119,7284,0,5023,3512,10419,3543
6,11688,3878,5951,21423,3102,5023,0,11557,6479,34261
7,2243,3202,5768,27342,1562,3512,11557,0,5615,7095
8,8857,6699,16578,51341,7180,10419,6479,5615,0,4448
9,7248,4198,4242,15826,1917,3543,34261,7095,4448,0


In [4]:
# Importing Dataset Cost
# This dataset consists of cost from each nodes
CAB_10_nodes_cost = pd.read_csv("10_nodes_CAB_cost.csv", delimiter= ";", header = None)
CAB_25_nodes_cost = pd.read_csv("25_nodes_CAB_cost.csv", delimiter= ";", header = None)
TR_55_nodes_cost = pd.read_csv("55_nodes_TR_cost.csv", delimiter= ";", header = None)
TR_81_nodes_cost = pd.read_csv("81_nodes_TR_cost.csv", delimiter= ";", header = None)
RGP_100_nodes_cost = pd.read_csv("100_nodes_RGP_cost.csv", delimiter= ";", header = None)

In [5]:
CAB_10_nodes_cost

,0,1,2,3,4,5,6,7,8,9
0,0.0000,576.9631,946.4954,597.5972,373.8127,559.7673,709.0215,1208.3280,603.6477,695.2080
1,576.9631,0.0000,369.5327,613.0386,429.1079,312.8831,1196.4890,1502.1400,405.8975,1241.9610
2,946.4954,369.5327,0.0000,858.3308,749.6018,556.0706,1541.2730,1764.7910,621.3306,1603.1650
3,597.5972,613.0386,858.3308,0.0000,255.0303,311.3071,790.1213,907.4331,237.0703,932.2173
4,373.8127,429.1079,749.6018,255.0303,0.0000,225.8954,794.1726,1080.3740,238.9440,879.5647
5,559.7673,312.8831,556.0706,311.3071,225.8954,0.0000,1009.6890,1216.8680,94.2588,1104.5740
6,709.0215,1196.4890,1541.2730,790.1213,794.1726,1009.6890,0.0000,663.8762,982.7378,221.4220
7,1208.3280,1502.1400,1764.7910,907.4331,1080.3740,1216.8680,663.8762,0.0000,1143.7910,874.5181
8,603.6477,405.8975,621.3306,237.0703,238.9440,94.2588,982.7378,1143.7910,0.0000,1094.9060
9,695.2080,1241.9610,1603.1650,932.2173,879.5647,1104.5740,221.4220,874.5181,1094.9060,0.0000


## Random Initial Solution and Population

In [6]:
# Random Initial Solution Function

def random_initial_solution (cost_matrix, number_hubs):
    
    number_nodes = cost_matrix.shape[0] # Number of Nodes is based on Cost Matrix Shape.
    nodes = range(1, number_nodes + 1) # Nodes is range from 1 to total nodes + 1
    
    hubs = random.sample(nodes, number_hubs) # Select the hubs randomly.
    spokes = [node for node in nodes if node not in hubs] # Spokes is node which is not in the hub list defined before.
    initial_solution = [0]*number_nodes # Initial Solution is array format with width total number of nodes. As an initial solution, the array consists of zero value.
    
    for node in nodes:
        if node in hubs: # If node is listed on hub list
            initial_solution [node - 1] = node # Then Initial Solution for Node - 1 equals with Node
        else:
            hub_spoke_cost = {hub : cost_matrix[node - 1][hub - 1] for hub in hubs}
            initial_solution[node - 1] = min(hub_spoke_cost, key = hub_spoke_cost.get)
    
    return initial_solution # Initial Solution for Hub Selection and Assignment

In [8]:
# Test Random Initial Function

test_solution_1 = random_initial_solution (CAB_10_nodes_cost, 3)

print ('Initial Solution Assigment: ', test_solution_1)
print ('Hub: ', set (test_solution_1))

Initial Solution Assigment:  [1, 6, 6, 4, 6, 6, 1, 4, 6, 1]
Hub:  {1, 4, 6}


In [9]:
# Initial Population based on Random Initial Solution

def initial_population (cost_matrix, n_hub, n_solution):
        # Generate Random Initial Solution and store into DataFrame
        df_random_solutions = {c: random_initial_solution (cost_matrix, n_hub)
                                for c in range(n_solution)}
    
        df_random_solutions = pd.DataFrame (df_random_solutions)

        initial_population = [tuple (list(df_random_solutions[i])) for i in range(0, n_solution)]
    
        in_population = [list(individual) for individual in initial_population]
        
        return in_population

In [10]:
# Test Population Function

initial_population_test1 = initial_population (CAB_25_nodes_cost, 3, 100)

In [11]:
# Test Population Function

initial_population_test2 = initial_population (CAB_10_nodes_cost, 3, 100)

In [12]:
# Flow loc function
# Flow loc function is used to get economic of scale based on piece wise

def flow_loc_function (flow):
    if flow < 0:
        raise ValueError ('No negative value')
    elif flow < 50000:
        unit_flow_cost = 0 + 1*flow
    elif flow < 100000:
        unit_flow_cost = 10000 + 0.8*flow
    elif flow < 200000:
        unit_flow_cost = 30000 + 0.6*flow
    elif flow >= 200000:
        unit_flow_cost = 70000 + 0.4*flow

    return unit_flow_cost

In [13]:
# Test Flow Loc Function

test_flow_loc1 = flow_loc_function (10000)

print ('Unit Flow Cost: ', test_flow_loc1)

Unit Flow Cost:  10000


## Cost Function

In [14]:
# Total Network cost function

def network_cost (solution, flow_matrix, cost_matrix):
    
    number_nodes = cost_matrix.shape[0]
    cost = 0 # Cost is initialized with zero value

    spoke_to_hub_cost = 0
    for node_1 in range(number_nodes): # For Node 1 in the range of number nodes
        for node_2 in range(number_nodes): # For Node 2 in the range of number nodes
            spoke_to_hub_cost += (flow_matrix[node_1][node_2] * (cost_matrix[node_1][solution[node_1]-1] + cost_matrix[solution[node_2]-1][node_2]))

    hub = set (solution) # Based on solution, check the unique number, which are the hubs number.

    hub_to_hub_cost = 0
    for hub_k in (hub): # Choose Hub K  
        for hub_m in (hub): # Choose Hub M
            if hub_k != hub_m: # Hub K not equals with Hub M
                hub_to_hub_flow = 0
                for node_1 in range (number_nodes): # For all Node 1 in range of number nodes
                    for node_2 in range (number_nodes): # For all Node 2 in range of number nodes
                        if node_1 != node_2:
                            if (solution[node_1] == hub_k) and (solution[node_2]) == hub_m:
                                hub_to_hub_flow += flow_matrix[node_1][node_2]

                hub_to_hub_cost += flow_loc_function (hub_to_hub_flow) * cost_matrix[hub_k-1][hub_m-1]
    
    cost = spoke_to_hub_cost + hub_to_hub_cost

    return cost

In [15]:
# Test Cost Network

test_cost1 = network_cost (test_solution_1, CAB_10_nodes_flow, CAB_10_nodes_cost)

print ('Total Cost Network: ', test_cost1)

Total Cost Network:  839824940.1342003


## Neighbourhood Structure (NS)

Neighbourhood structure is needed to be defined in the case of heuristic, including Genetic Algorithm. Neighboorhood Structure is used during Mutation Operator phase. There are several methods of mutation operator, such as Shift and Exchange Methods.

In [16]:
# Neighbourhood Structure 1: Reallocate Hub - Swap Hub (Random Hub to Non Hub (Spoke), and Random Non Hub (Spoke) to Hub)
    # Paper 1: Solution Algorithm for the capacitated single allocation hub location problem - 1999
    # Paper 2: Efficient Algorithm for the uncapacitated single allocation p-hub median problem - 1996
    # Paper 3: Efficient Simulated Annealing based solution approaches to the competitive single and multiple allocation hub location problem

def NS1 (solution, cost_matrix):

    node = cost_matrix.shape[0]

    hub_to_spoke = random.choice (list(set(solution))) # Selecting randomly from solution list (hub list) to be a spoke.

    spokes = [node for node in range(1, node+1) if node not in set(solution)] # Spoke is in the range 1 to node+1 if it is not in solution list (hub list).

    spoke_to_hub = random.choice (spokes) # Selecting randomly spokes (non hub) from spoke list (spokes) to be a hub.   

    solution = [spoke_to_hub if x == hub_to_spoke else x for x in solution] # Solution is  new hub (from spoke) if x equals with new spoke (from hub), else it is x for x in solution.

    solution[spoke_to_hub-1] = spoke_to_hub # Solution (Spoke to hub - 1) equals with new hub (from spoke).

    return solution

In [17]:
# Neighbourhood Structure 1

solution_before_NS1 = [21, 21, 3, 6, 21, 6, 6, 6, 6, 6, 6, 6, 3, 21, 6, 6, 21, 21, 3, 21, 21, 6, 6, 21, 21]

solution_after_NS1 = NS1 (solution_before_NS1, CAB_25_nodes_cost)

print ('Solution Before NS1', solution_before_NS1 )
print ('Solution After NS1', solution_after_NS1 )

Solution Before NS1 [21, 21, 3, 6, 21, 6, 6, 6, 6, 6, 6, 6, 3, 21, 6, 6, 21, 21, 3, 21, 21, 6, 6, 21, 21]
Solution After NS1 [16, 16, 3, 6, 16, 6, 6, 6, 6, 6, 6, 6, 3, 16, 6, 16, 16, 16, 3, 16, 16, 6, 6, 16, 16]


In [18]:
# Neighbourhood Structure 2: Swap Nodes: Swap the allocations of two (randomly chosen) non hub nodes from different cluster.
    # Paper 1: Solution Algorithm for the capacitated single allocation hub location problem - 1999
    # Paper 2: Mutation Procedure - Solving the uncapacitated hub location problem using genetic algorithm - 2005
    # Paper 3: Using simulated annealing to solve the p-hub median problem

def NS2 (solution, cost_matrix):

    node = cost_matrix.shape[0]

    hubs = list(set(solution)) # Hubs is based on list of solution.

    hubs_chosen = random.sample (hubs, 2) # Select randomly based on list of solution. 2 is number of items to be returned.

    hub_spoke  = {}
    for hub in set(solution):
        hub_spoke[hub] = [i+1 for i,val in enumerate(solution) if val==hub]
        for spokes in list(hub_spoke.values()):
            hub_spoke[hub] = [index for index in spokes if index != hub-1] #hubs and their spokes' indexes.

    l1 = hub_spoke[hubs_chosen[0]] # l1 is hub chosen first.
    l2 = hub_spoke[hubs_chosen[1]] # l2 is hub chosed second.
    
    spoke1 = random.choice(l1)
    spoke2 = random.choice(l2)

    solution[spoke1-1] = hubs_chosen[1]
    solution[spoke2-1] = hubs_chosen[0]
    
    solution[hubs_chosen[0]-1] = hubs_chosen[0]
    solution[hubs_chosen[1]-1] = hubs_chosen[1]
    
    return solution

In [19]:
# Test Neighbourhood Structure 2

solution_before_NS2 = [21, 21, 3, 6, 21, 6, 6, 6, 6, 6, 6, 6, 3, 21, 6, 6, 21, 21, 3, 21, 21, 6, 6, 21, 21]

print ('Solution before NS2', solution_before_NS2)

solution_after_NS2 = NS2 (solution_before_NS2, CAB_25_nodes_cost)

print ('Solution after NS2', solution_after_NS2)

Solution before NS2 [21, 21, 3, 6, 21, 6, 6, 6, 6, 6, 6, 6, 3, 21, 6, 6, 21, 21, 3, 21, 21, 6, 6, 21, 21]
Solution after NS2 [21, 21, 3, 6, 21, 6, 6, 6, 6, 6, 6, 6, 3, 21, 6, 6, 21, 21, 3, 21, 21, 6, 21, 21, 6]


In [20]:
# Neighbourhood Structure 3: Swapping hubs - Swaping the nodes of two hubs

def NS3 (solution, cost_matrix):

    node = cost_matrix.shape[0]

    hubs = list(set(solution))
    hubs_chosen = random.sample(hubs, 2)

    hub_spoke = {}
    for hub in set(solution):
        hub_spoke[hub] = [i for i,val in enumerate(solution) if val==hub]#it takes the idexes of the hubs in the solution
        for spokes in list(hub_spoke.values()):
            hub_spoke[hub] = [index for index in spokes if index != hub-1] #hubs and their spokes' indexes 
    
    for i in hub_spoke[hubs_chosen[0]]:
        solution[i] = hubs_chosen[1]
    for i in hub_spoke[hubs_chosen[1]]:
        solution[i] = hubs_chosen[0]
    
    return solution

In [21]:
# Test Neighbourhood Structure 3

solution_before_NS3 = [21, 21, 3, 6, 21, 6, 6, 6, 6, 6, 6, 6, 3, 21, 6, 6, 21, 21, 3, 21, 21, 6, 6, 21, 21]

print ('Solution before NS3', solution_before_NS3)

solution_after_NS3 = NS3 (solution_before_NS3, CAB_25_nodes_cost)

print ('Solution after NS3', solution_after_NS3)

Solution before NS3 [21, 21, 3, 6, 21, 6, 6, 6, 6, 6, 6, 6, 3, 21, 6, 6, 21, 21, 3, 21, 21, 6, 6, 21, 21]
Solution after NS3 [6, 6, 3, 21, 6, 6, 21, 21, 21, 21, 21, 21, 3, 6, 21, 21, 6, 6, 3, 6, 21, 21, 21, 6, 6]


In [22]:
# Neighbourhood Structure 4: Reallocate a node - Change the allocation of a randomly chosen non hub node to a different randomly chosen hub
    # Paper 1: Solution Algorithm for the capacitated single allocation hub location problem - 1999
    # Paper 2: Efficient Algorithm for the uncapacitated single allocation p-hub median problem - 1996
    # Paper 3: Efficient Simulated Annealing based solution approaches to the competitive single and multiple allocation hub location problem
    # Paper 4: Mutation Procedure - Solving the uncapacitated hub location problem using genetic algorithm - 2005

def NS4 (solution, cost_matrix):

    node = cost_matrix.shape[0]

    hubs = list(set(solution))
    nodes = list(range(1,node+1))
    for hub in hubs:
        nodes.remove(hub)

    spoke = random.choice(nodes)

    hub1 = solution[spoke-1]
    hubs.remove(hub1)
    hub2 = random.choice(hubs)

    solution[spoke-1] = hub2
    
    return solution

In [23]:
solution_before_NS4 = [21, 21, 3, 6, 21, 6, 6, 6, 6, 6, 6, 6, 3, 21, 6, 6, 21, 21, 3, 21, 21, 6, 6, 21, 21]

print ('Solution before NS4', solution_before_NS4)

solution_after_NS4 = NS4 (solution_before_NS4, CAB_25_nodes_cost)

print ('Solution after NS4', solution_after_NS4)

Solution before NS4 [21, 21, 3, 6, 21, 6, 6, 6, 6, 6, 6, 6, 3, 21, 6, 6, 21, 21, 3, 21, 21, 6, 6, 21, 21]
Solution after NS4 [21, 21, 3, 6, 21, 6, 6, 21, 6, 6, 6, 6, 3, 21, 6, 6, 21, 21, 3, 21, 21, 6, 6, 21, 21]


## Genetic Algorithm

In Genetic Algorithm, there are several things to be considered, which are Fitness, Operator Selection Methods, Cross Over & Off Spring, and Mutation.

### Chromosome Fitness
Fitness of model can be fined by the cost

In [24]:
# Chromosome Fitness
def fitness_evaluation (population, flow_matrix, cost_matrix):
    
    solution_cost = [] # Creating an array
    
    for solution in population: # For all chromosome/solution in population
        cost = network_cost (solution, flow_matrix, cost_matrix)
        solution_cost.append((cost, solution))
 
    return solution_cost

### Operator Selection Methods
- There are several operator selection methods, such as Roulette Wheel Selection, Boltzman Selection, Tournament Selection, and Rank Selection
- Operator Selection is used to replace the lowest one with the best one.
- Tournament Selection will select the individuals with uniform probability with replacement.

In [27]:
# Tournament Selection Function

def tournament_selection (fitness_values_and_solutions, T_Individual): 

    chosen_individuals1 = random.sample (fitness_values_and_solutions, T_Individual) 
    # Random select solution which contains fintess value as defined on fintess evaluation function.
    # T is total individuals selected.

    parent1_fitness = min([individual[0] for individual in chosen_individuals1]) # Parent 1 Fitness is selected based on the minimum fitness (the best one or minimum cost)

    for individual in chosen_individuals1:
        if individual[0] == parent1_fitness:
            parent1 = individual

    chosen_individuals2 = random.sample(fitness_values_and_solutions, T_Individual)
    # Random select solution which contains fintess value as defined on fintess evaluation function.
    # T is total individuals selected
    
    parent2_fitness = min([individual[0] for individual in chosen_individuals2]) # Parent 2 Fitness is selected based on the minimum fitness (the best one or minimum cost)
    
    for individual in chosen_individuals2:
        if individual[0] == parent2_fitness:
            parent2 = individual
            
    parents = [parent[1] for parent in [parent1, parent2] ]
    
    return parents

### Cross Over and Offspring

- In order to create new generation, we need to cross over in cross over point. It can be single point cross over, two point cross over, multiple point cross over, or uniform cross over.
- There are two terminologies which are Parents and Off Spring. Parent is the initial solution and off spring is the new solution.
- Cross over will combine two parents, which will result into two off springs.
- Cross over wont guarantee a fitter solution.
- While doing cross over, we need to decide the cross over point.

In [28]:
# Cross-Over Function

def crossover (parents, cost_matrix):

    node = cost_matrix.shape[0] # Number of Nodes is based on Cost Matrix Shape.

    # Off spring
    offspring = [0]*node # Offspring zero array at first.

    # Random Point for Cross Over
    random_point = random.choice (range(0, node)) # Using randomization while choosing cross over point. The range is the total node (array).
    
    # Chosing parent 1
    parent1 = parents[0] # Selecting random parents as parent1 based on parents list as defined.

    for i in range (0, random_point+1): # For all number from index 0 to random point (Decided above) + 1 then
        offspring.pop(i) # remove values in the range 
        offspring.insert (i, parent1[i]) # then insert in the position i with parent1 with index i.
 
    # Chosing parent 2
    parent2 = parents[1] 
            
    for i in range (random_point, node): # For all number from index 0 to random point (Decided above) + 1 then
        offspring.pop (i) # remove values in the range
        offspring.insert (i, parent2[i]) # then insert in the position i with parent1 with index i.
    
    return offspring # Offspring is the new chromosome based on crossover two parents.

### Offspring Repair or Rearrangement

- Offspring Rearrangement is needed for Assign Array if the hub chosen is not valid anymore.
- Rearrangement new hub is based on distance matrix.

In [30]:
# Offspring Repair or Rearrangement Function

def offspring_repair (offspring, cost_matrix,  n_hub): #of if len(set(offspring)) > n_hub in the main function
 
    node = cost_matrix.shape[0]

    hubs_offspring = []
    
    if len(set(offspring)) == n_hub:
        for hub in set(offspring):
            offspring[hub-1] = hub
    
    elif len(set(offspring)) > n_hub:  
        for i in range (1500) :
            if len(set(offspring)) == n_hub:
                break
            for hub in set(offspring):
                for i in range(0, node):
                    if offspring[i] not in hubs_offspring:
                        hubs_offspring.append(hub)                

            hub_spoke_indexes = {}

            for hub in hubs_offspring:
                hub_spoke_indexes[hub] = [i for i,val in enumerate(offspring) if val == hub] #takes the indexes of the hubs in the string
            hub_to_remove = hubs_offspring.pop(-1)
            
            hubs = list(set(hubs_offspring))
            chosen_hub = random.choice(hubs)
            
            for index in hub_spoke_indexes[hub_to_remove]:
                offspring.pop(index)
                offspring.insert(index, chosen_hub)
            for hub in hubs:
                offspring.pop(hub-1)
                offspring.insert(hub-1, hub)

    return offspring

In [33]:
# Feasibility Offspring

def feasibility_offspring (offspring, n_hubs):
    
    if len(set(offspring)) == n_hubs: # and offspring != list(parents[0]) and offspring != list(parents[1]):
        return 1
    else:
        return 0

### Mutation

- Mutation is executed for offspring. 
- Mutation will replace one of value in solution with another value. For example: Initial : 001001, Replace second with "1", then 011001 or using neighbourhood structure.
- There are two types mutations example: Shift and Exchange.
    - Shift is to select a spoke and reassign to new hub randomly. If there is only one hub in the string, then this method is not applicable.
    - Exchange  is  to select two spokes and switch the hub or assignments. If there is only one hub or one spoke, then this method is not applicable.

In [31]:
# Mutation Function

def mutation (offspring, cost_matrix, NS_type):

    # operation_list = ["NS4", "NS3", "NS2", "NS1"] # Neighbourhood structures have been defined before. In this case, we choose NS4, NS3, and NS1 as neighbourhood structure.

    mutation_procedure = NS_type # Operation method are chosen randomly based on operation list.

    if mutation_procedure == "NS4":
        return NS4 (offspring, cost_matrix) # If operation method select NS4 then execute NS4 function.

    elif mutation_procedure == "NS3":
        return NS4 (offspring, cost_matrix) # If operation method select NS3 then execute NS3 function. 
    
    elif mutation_procedure == "NS2":
        return NS2 (offspring, cost_matrix) # If operation method select NS3 then execute NS2 function.
    
    else:
        return NS1 (offspring, cost_matrix) # If operation method select NS1 then execute NS1 function.

In [32]:
# Genetic Algorithm Function

def genetic_algorithm (n_hub, T_Individual, flow_matrix, cost_matrix, n_solution, iter):
    
    start = time.time()
    offsprings = []
    
    # The initial population using least cost initial population defined above
    init_pop = initial_population (cost_matrix, n_hub, n_solution)
    
    # Population Evaluation using fitness evaluation function defined above
    pop_evaluation = fitness_evaluation (init_pop, flow_matrix, cost_matrix)
    
    best_cost = float ('inf')
    best_solution = None

    counter = 0
    while counter < iter:
        # Tournament Selection
        parents = tournament_selection (pop_evaluation, T_Individual) # Choose parent as operator selection using Tournament Selection
        
        total_offsprings = 0
        while total_offsprings < n_solution:
            
            # Cross Over, Offspring Repairement and New Fitness Evaluation
            offspring_crossovered = crossover (parents, cost_matrix)
            offspring_crossovered_fixed = offspring_repair (offspring_crossovered, cost_matrix, n_hub)
            evaluation_c = network_cost (offspring_crossovered_fixed, flow_matrix, cost_matrix)
                
            # Mutation Process - Using two type of mutation process NS1 and NS2
            # Source: Solving the uncapacitated hub location problem using genetic algorithms

            operator_selected = random.choice(['NS1', 'NS2'])
            offspring_crossovered_mutated_final = mutation (offspring_crossovered_fixed, cost_matrix, operator_selected) 
            evaluation_cm_final = network_cost (offspring_crossovered_mutated_final, flow_matrix, cost_matrix)

            if evaluation_cm_final < evaluation_c: # If Fitness Evaluation of Mutation is below (better) than before than mutation (only cross-over), than accept
                final_offspring = offspring_crossovered_mutated_final
                final_offspring_cost = evaluation_cm_final
                offsprings.append ((final_offspring_cost, final_offspring))
            else:
                final_offspring = offspring_crossovered_fixed
                final_offspring_cost = evaluation_c
                offsprings.append ((final_offspring_cost, final_offspring))
            
            if final_offspring_cost < best_cost:
                best_solution = final_offspring.copy()
                best_cost = final_offspring_cost
            else:
                best_solution = best_solution
                best_cost = best_cost
            total_offsprings +=1

        # Combine with Initial Population Evaluation + Offspring, sorted, and take n solution
        initpopeval_and_offsprings = offsprings + pop_evaluation
        initpopeval_and_offsprings.sort()
        initpopeval_and_offsprings = initpopeval_and_offsprings[0:n_solution]

        # Pop Evaluation for next iteration
        pop_evaluation = initpopeval_and_offsprings # Initial Population = Offsprings
        offsprings = []
        
        counter +=1 # Add counter for iteration

    end = time.time()

    return best_cost, best_solution, end-start, (end-start)/iter

## Simulation for Several Cases

In [33]:
# Simulate Genetic Algorithm with CAB 10 Dataset
total_running = 10
minimum_value_cab10_3_ga = []

for i in range(total_running):
    random.seed(i+100)
    result = genetic_algorithm (3, 10, CAB_10_nodes_flow, CAB_10_nodes_cost, 100, 50)
    minimum_value_cab10_3_ga.append(result)

minimum_value_dataframe_cab10_3_ga = pd.DataFrame (minimum_value_cab10_3_ga, columns=["Minimum Cost", "Allocation", "Computational Time", "Computational Time per iteration"])
minimum_value_dataframe_cab10_3_ga

,Minimum Cost,Allocation,Computational Time,Computational Time per iteration
0,7.409994e+08,"[4, 9, 9, 4, 4, 9, 7, 4, 9, 7]",7.970639,0.159413
1,7.409994e+08,"[4, 9, 9, 4, 4, 9, 7, 4, 9, 7]",8.000740,0.160015
2,7.409994e+08,"[4, 9, 9, 4, 4, 9, 7, 4, 9, 7]",7.997889,0.159958
3,7.409994e+08,"[4, 9, 9, 4, 4, 9, 7, 4, 9, 7]",7.948538,0.158971
4,7.409994e+08,"[4, 9, 9, 4, 4, 9, 7, 4, 9, 7]",7.988077,0.159762
5,7.409994e+08,"[4, 9, 9, 4, 4, 9, 7, 4, 9, 7]",7.999285,0.159986
6,7.409994e+08,"[4, 9, 9, 4, 4, 9, 7, 4, 9, 7]",7.958285,0.159166
7,7.409994e+08,"[4, 9, 9, 4, 4, 9, 7, 4, 9, 7]",7.976393,0.159528
8,7.409994e+08,"[4, 9, 9, 4, 4, 9, 7, 4, 9, 7]",7.988406,0.159768
9,7.409994e+08,"[4, 9, 9, 4, 4, 9, 7, 4, 9, 7]",7.977899,0.159558


In [114]:
minimum_value_dataframe_cab10_3_ga.to_csv ('minimum_value_dataframe_cab10_3_ga_50iter.csv')

In [115]:
# Simulate Genetic Algorithm with CAB 25 Dataset - 3 Hubs
total_running = 10
minimum_value_cab25_3_ga = []

for i in range(total_running):
    random.seed(i+100)
    result = genetic_algorithm (3, 10, CAB_25_nodes_flow, CAB_25_nodes_cost, 100, 50)
    minimum_value_cab25_3_ga.append(result)

minimum_value_dataframe_cab25_3_ga = pd.DataFrame (minimum_value_cab25_3_ga, columns=["Minimum Cost", "Allocation", "Computational Time", "Computational Time per iteration"])
minimum_value_dataframe_cab25_3_ga

,Minimum Cost,Allocation,Computational Time,Computational Time per iteration
0,8.374475e+09,"[4, 18, 18, 4, 4, 4, 4, 4, 4, 4, 4, 12, 4, 18,...",48.552066,0.971041
1,8.478429e+09,"[5, 17, 17, 5, 5, 5, 5, 5, 5, 5, 5, 12, 5, 5, ...",47.978494,0.959570
2,8.466881e+09,"[4, 18, 18, 4, 4, 4, 4, 4, 4, 4, 4, 12, 4, 18,...",48.630149,0.972603
3,8.374475e+09,"[4, 18, 18, 4, 4, 4, 4, 4, 4, 4, 4, 12, 4, 18,...",48.631926,0.972639
4,8.446904e+09,"[21, 2, 2, 21, 21, 2, 21, 21, 2, 21, 21, 12, 2...",48.624389,0.972488
5,8.446904e+09,"[21, 2, 2, 21, 21, 2, 21, 21, 2, 21, 21, 12, 2...",48.514004,0.970280
6,8.465824e+09,"[2, 2, 2, 21, 21, 2, 21, 21, 2, 21, 21, 12, 21...",48.249797,0.964996
7,8.374475e+09,"[4, 18, 18, 4, 4, 4, 4, 4, 4, 4, 4, 12, 4, 18,...",48.402100,0.968042
8,8.374475e+09,"[4, 18, 18, 4, 4, 4, 4, 4, 4, 4, 4, 12, 4, 18,...",48.365269,0.967305
9,8.374475e+09,"[4, 18, 18, 4, 4, 4, 4, 4, 4, 4, 4, 12, 4, 18,...",48.775592,0.975512


In [119]:
minimum_value_dataframe_cab25_3_ga.to_csv ('minimum_value_dataframe_cab25_3_ga_50iter.csv')

In [117]:
# Simulate Genetic Algorithm with CAB 25 Dataset - 5 Hubs
total_running = 10
minimum_value_cab25_5_ga = []

for i in range(total_running):
    random.seed(i+100)
    result = genetic_algorithm (5, 10, CAB_25_nodes_flow, CAB_25_nodes_cost, 100, 50)
    minimum_value_cab25_5_ga.append(result)

minimum_value_dataframe_cab25_5_ga = pd.DataFrame (minimum_value_cab25_5_ga, columns=["Minimum Cost", "Allocation", "Computational Time", "Computational Time per iteration"])
minimum_value_dataframe_cab25_5_ga

,Minimum Cost,Allocation,Computational Time,Computational Time per iteration
0,7.920597e+09,"[4, 17, 17, 4, 5, 4, 4, 4, 4, 4, 4, 12, 4, 14,...",54.319526,1.086391
1,7.829156e+09,"[1, 17, 17, 4, 4, 4, 4, 4, 4, 1, 4, 12, 1, 1, ...",53.767734,1.075355
2,7.929310e+09,"[1, 17, 17, 4, 4, 6, 4, 12, 6, 1, 4, 12, 1, 1,...",54.151694,1.083034
3,7.817837e+09,"[1, 17, 17, 4, 4, 4, 4, 4, 4, 4, 4, 12, 4, 14,...",54.183060,1.083661
4,7.773952e+09,"[1, 17, 17, 4, 4, 4, 4, 8, 4, 1, 4, 12, 1, 1, ...",53.966854,1.079337
5,7.695627e+09,"[1, 17, 17, 4, 4, 4, 7, 7, 4, 7, 4, 12, 1, 1, ...",54.390525,1.087810
6,7.806341e+09,"[1, 17, 17, 4, 4, 4, 4, 8, 4, 4, 4, 12, 4, 1, ...",54.005660,1.080113
7,7.675539e+09,"[13, 17, 17, 4, 4, 4, 13, 4, 4, 13, 4, 12, 13,...",53.763500,1.075270
8,7.816650e+09,"[21, 17, 17, 21, 6, 6, 21, 21, 6, 21, 21, 12, ...",53.955745,1.079115
9,7.817837e+09,"[1, 17, 17, 4, 4, 4, 4, 4, 4, 4, 4, 12, 4, 14,...",53.365086,1.067302


In [120]:
minimum_value_dataframe_cab25_5_ga.to_csv ('minimum_value_dataframe_cab25_5_ga_50iter.csv')

In [63]:
# Simulate Genetic Algorithm with TR 55 dataset - 3 Hubs
total_running = 10
minimum_value_tr55_3_ga = []

for i in range(total_running):
    random.seed(i+100)
    result = genetic_algorithm (3, 10, TR_55_nodes_flow, TR_55_nodes_cost, 100, 10)
    minimum_value_tr55_3_ga.append(result)

minimum_value_dataframe_tr55_3_ga = pd.DataFrame (minimum_value_tr55_3_ga, columns=["Minimum Cost", "Allocation", "Computational Time", "Computational Time per iteration"])
minimum_value_dataframe_tr55_3_ga

,Minimum Cost,Allocation,Computational Time,Computational Time per iteration
0,2.885834e+10,"[34, 34, 34, 4, 30, 34, 30, 30, 34, 34, 30, 30...",49.635384,4.963538
1,2.830498e+10,"[19, 19, 19, 4, 4, 4, 19, 30, 19, 19, 4, 30, 3...",52.890462,5.289046
2,2.964513e+10,"[45, 17, 17, 45, 45, 17, 30, 30, 17, 17, 30, 3...",48.884121,4.888412
3,3.051571e+10,"[45, 42, 42, 30, 45, 42, 30, 30, 42, 42, 30, 3...",52.955965,5.295596
4,2.909469e+10,"[45, 15, 15, 45, 45, 15, 30, 30, 15, 15, 45, 3...",51.577059,5.157706
5,3.016070e+10,"[34, 34, 34, 45, 45, 34, 12, 12, 34, 34, 12, 1...",51.590377,5.159038
6,3.031076e+10,"[1, 1, 45, 45, 1, 45, 30, 30, 45, 1, 30, 30, 3...",50.289469,5.028947
7,3.035807e+10,"[55, 55, 4, 4, 18, 4, 18, 18, 55, 55, 18, 18, ...",50.755207,5.075521
8,3.083597e+10,"[28, 28, 28, 4, 4, 28, 30, 30, 28, 28, 30, 30,...",48.831961,4.883196
9,2.938468e+10,"[55, 55, 55, 4, 4, 55, 30, 30, 55, 55, 4, 30, ...",50.457245,5.045724


In [65]:
minimum_value_dataframe_tr55_3_ga.to_csv ('minimum_value_dataframe_tr55_3_ga_10iter.csv')

In [66]:
# Simulate Genetic Algorithm with TR 55 dataset - 5 Hubs
total_running = 10
minimum_value_tr55_5_ga = []

for i in range(total_running):
    random.seed(i+100)
    result = genetic_algorithm (5, 10, TR_55_nodes_flow, TR_55_nodes_cost, 100, 10)
    minimum_value_tr55_5_ga.append(result)

minimum_value_dataframe_tr55_5_ga = pd.DataFrame (minimum_value_tr55_5_ga, columns=["Minimum Cost", "Allocation", "Computational Time", "Computational Time per iteration"])
minimum_value_dataframe_tr55_5_ga

,Minimum Cost,Allocation,Computational Time,Computational Time per iteration
0,2.704058e+10,"[1, 42, 17, 4, 12, 17, 12, 12, 17, 17, 12, 12,...",58.208568,5.820857
1,2.587997e+10,"[55, 55, 17, 4, 4, 17, 30, 30, 17, 17, 4, 30, ...",62.903270,6.290327
2,2.577154e+10,"[34, 34, 34, 4, 31, 34, 8, 8, 34, 34, 31, 8, 2...",57.043277,5.704328
3,2.752401e+10,"[34, 34, 34, 18, 24, 34, 27, 18, 34, 34, 24, 1...",51.943508,5.194351
4,2.545074e+10,"[1, 1, 15, 4, 33, 15, 33, 4, 15, 15, 4, 26, 33...",54.052161,5.405216
5,2.688974e+10,"[1, 42, 42, 29, 31, 42, 12, 12, 42, 42, 31, 12...",63.315883,6.331588
6,2.564508e+10,"[19, 19, 10, 4, 33, 10, 33, 26, 10, 10, 33, 26...",67.123269,6.712327
7,2.674632e+10,"[25, 15, 15, 4, 25, 15, 13, 13, 15, 15, 4, 26,...",68.318818,6.831882
8,2.587206e+10,"[19, 19, 19, 4, 33, 4, 33, 12, 19, 19, 33, 12,...",57.872904,5.787290
9,2.803270e+10,"[25, 2, 41, 45, 45, 41, 30, 30, 41, 41, 45, 30...",64.791320,6.479132


In [67]:
minimum_value_dataframe_tr55_5_ga.to_csv ('minimum_value_dataframe_tr55_5_ga_10iter.csv')

In [68]:
# Simulate Genetic Algorithm with TR 81 dataset - 5 Hubs
total_running = 10
minimum_value_tr81_5_ga = []

for i in range(total_running):
    random.seed(i+100)
    result = genetic_algorithm (5, 10, TR_81_nodes_flow, TR_81_nodes_cost, 100, 10)
    minimum_value_tr81_5_ga.append(result)

minimum_value_dataframe_tr81_5_ga = pd.DataFrame (minimum_value_tr81_5_ga, columns=["Minimum Cost", "Allocation", "Computational Time", "Computational Time per iteration"])
minimum_value_dataframe_tr81_5_ga

,Minimum Cost,Allocation,Computational Time,Computational Time per iteration
0,4.575982e+10,"[46, 46, 3, 46, 60, 6, 3, 60, 3, 3, 3, 46, 46,...",125.796253,12.579625
1,4.938766e+10,"[68, 68, 68, 60, 60, 68, 45, 60, 45, 45, 54, 4...",160.797833,16.079783
2,4.818533e+10,"[27, 27, 6, 27, 6, 6, 35, 27, 35, 16, 16, 27, ...",188.580255,18.858025
3,4.871075e+10,"[70, 2, 71, 2, 71, 71, 70, 2, 16, 16, 16, 2, 2...",126.773222,12.677322
4,4.862691e+10,"[23, 23, 64, 25, 6, 6, 64, 25, 64, 64, 54, 23,...",146.848104,14.684810
5,4.700814e+10,"[66, 46, 64, 69, 66, 66, 64, 69, 64, 64, 34, 4...",113.901183,11.390118
6,4.697127e+10,"[80, 80, 54, 58, 58, 18, 20, 58, 20, 54, 54, 5...",169.901921,16.990192
7,4.875441e+10,"[66, 23, 6, 23, 66, 6, 6, 23, 16, 16, 16, 23, ...",150.447719,15.044772
8,4.726514e+10,"[66, 23, 43, 23, 66, 6, 43, 23, 43, 43, 43, 23...",166.026163,16.602616
9,4.895686e+10,"[50, 21, 16, 21, 50, 50, 35, 28, 35, 16, 16, 2...",153.153084,15.315308


In [69]:
minimum_value_dataframe_tr81_5_ga.to_csv ('minimum_value_dataframe_tr81_5_ga_10iter.csv')

In [70]:
# Simulate Genetic Algorithm with TR 81 dataset - 7 Hubs
total_running = 10
minimum_value_tr81_7_ga = []

for i in range(total_running):
    random.seed(i+100)
    result = genetic_algorithm (7, 10, TR_81_nodes_flow, TR_81_nodes_cost, 100, 10)
    minimum_value_tr81_7_ga.append(result)

minimum_value_dataframe_tr81_7_ga = pd.DataFrame (minimum_value_tr81_7_ga, columns=["Minimum Cost", "Allocation", "Computational Time", "Computational Time per iteration"])
minimum_value_dataframe_tr81_7_ga

,Minimum Cost,Allocation,Computational Time,Computational Time per iteration
0,4.375714e+10,"[1, 1, 11, 69, 6, 6, 35, 69, 35, 35, 11, 44, 3...",236.687476,23.668748
1,4.630877e+10,"[27, 27, 7, 5, 5, 71, 7, 5, 35, 35, 81, 27, 27...",255.981803,25.598180
2,4.473414e+10,"[33, 33, 35, 23, 71, 71, 35, 61, 35, 35, 34, 2...",158.169329,15.816933
3,4.521592e+10,"[38, 23, 42, 23, 57, 6, 42, 23, 9, 9, 34, 23, ...",270.552090,27.055209
4,4.426303e+10,"[80, 55, 3, 21, 50, 3, 3, 21, 45, 45, 3, 21, 2...",138.042959,13.804296
5,4.657412e+10,"[1, 72, 64, 36, 60, 41, 64, 36, 64, 64, 41, 72...",244.777022,24.477702
6,4.483762e+10,"[68, 44, 26, 58, 6, 6, 20, 58, 20, 26, 26, 44,...",188.387283,18.838728
7,4.515486e+10,"[51, 44, 64, 28, 28, 81, 64, 28, 64, 64, 54, 4...",167.270096,16.727010
8,4.491082e+10,"[80, 80, 15, 49, 58, 38, 15, 49, 15, 10, 41, 4...",252.964949,25.296495
9,4.447090e+10,"[46, 46, 32, 13, 66, 66, 32, 12, 32, 45, 34, 1...",220.954408,22.095441


In [71]:
minimum_value_dataframe_tr81_7_ga.to_csv ('minimum_value_dataframe_tr81_7_ga_10iter.csv')

In [72]:
# Simulate Genetic Algorithm with RGP 100 dataset - 7 Hubs
total_running = 10
minimum_value_rgp100_7_ga = []

for i in range(total_running):
    random.seed(i+100)
    result = genetic_algorithm (7, 10, RGP_100_nodes_flow, RGP_100_nodes_cost, 100, 10)
    minimum_value_rgp100_7_ga.append(result)

minimum_value_dataframe_rgp100_7_ga = pd.DataFrame (minimum_value_rgp100_7_ga, columns=["Minimum Cost", "Allocation", "Computational Time", "Computational Time per iteration"])
minimum_value_dataframe_rgp100_7_ga

,Minimum Cost,Allocation,Computational Time,Computational Time per iteration
0,1.450288e+11,"[5, 5, 89, 55, 5, 61, 5, 89, 94, 89, 52, 70, 9...",483.208621,48.320862
1,1.439116e+11,"[1, 1, 1, 20, 5, 20, 20, 20, 20, 35, 82, 42, 4...",550.476889,55.047689
2,1.439910e+11,"[5, 48, 49, 4, 5, 49, 64, 64, 64, 5, 95, 4, 4,...",418.544251,41.854425
3,1.434493e+11,"[58, 84, 75, 58, 74, 49, 58, 75, 9, 58, 75, 49...",193.338311,19.333831
4,1.438236e+11,"[5, 36, 36, 36, 5, 5, 5, 64, 10, 10, 11, 11, 3...",279.685920,27.968592
5,1.426072e+11,"[5, 5, 88, 4, 5, 49, 47, 49, 49, 47, 27, 4, 27...",324.112388,32.411239
6,1.419703e+11,"[1, 1, 1, 20, 1, 20, 76, 20, 20, 1, 11, 11, 1,...",308.738426,30.873843
7,1.442953e+11,"[92, 2, 21, 21, 48, 21, 99, 21, 17, 54, 2, 99,...",439.162777,43.916278
8,1.435892e+11,"[24, 24, 15, 52, 15, 44, 73, 24, 44, 15, 52, 9...",493.854144,49.385414
9,1.434949e+11,"[34, 5, 26, 29, 5, 34, 76, 34, 45, 71, 34, 34,...",193.282140,19.328214


In [74]:
minimum_value_dataframe_rgp100_7_ga.to_csv ('minimum_value_dataframe_rgp100_7_ga_10iter.csv')

In [110]:
# Simulate Genetic Algorithm with RGP 100 dataset - 10 Hubs
total_running = 10
minimum_value_rgp100_10_ga = []

for i in range(total_running):
    random.seed(i+100)
    result = genetic_algorithm (10, 10, RGP_100_nodes_flow, RGP_100_nodes_cost, 100, 10)
    minimum_value_rgp100_10_ga.append(result)

minimum_value_dataframe_rgp100_10_ga = pd.DataFrame (minimum_value_rgp100_10_ga, columns=["Minimum Cost", "Allocation", "Computational Time", "Computational Time per iteration"])
minimum_value_dataframe_rgp100_10_ga

,Minimum Cost,Allocation,Computational Time,Computational Time per iteration
0,1.433662e+11,"[34, 24, 52, 27, 51, 43, 33, 24, 33, 43, 27, 3...",694.574068,69.457407
1,1.423785e+11,"[1, 86, 33, 20, 5, 20, 33, 10, 9, 10, 86, 73, ...",612.730124,61.273012
2,1.433145e+11,"[1, 1, 98, 20, 96, 20, 20, 96, 20, 54, 77, 98,...",703.310839,70.331084
3,1.418440e+11,"[72, 2, 8, 4, 5, 96, 87, 8, 96, 87, 41, 4, 94,...",802.891994,80.289199
4,1.438872e+11,"[1, 36, 36, 55, 36, 33, 33, 53, 33, 13, 11, 79...",828.876548,82.887655
5,1.425280e+11,"[34, 86, 15, 20, 60, 20, 33, 30, 33, 15, 86, 3...",318.065284,31.806528
6,1.415336e+11,"[73, 17, 25, 20, 27, 20, 73, 20, 32, 25, 27, 2...",659.451510,65.945151
7,1.437850e+11,"[1, 1, 59, 20, 64, 20, 64, 20, 9, 83, 52, 64, ...",735.991766,73.599177
8,1.425005e+11,"[47, 17, 9, 4, 90, 97, 33, 94, 9, 90, 90, 4, 9...",713.649599,71.364960
9,1.414552e+11,"[79, 79, 8, 20, 64, 20, 33, 8, 9, 39, 79, 79, ...",404.889259,40.488926


In [111]:
minimum_value_dataframe_rgp100_10_ga.to_csv ('minimum_value_dataframe_rgp100_10_ga_10iter.csv')

## Sensitivity Analysis

### Sensitivity T Individual for Tournament Selection

In [121]:
# Simulate Genetic Algorithm with CAB 25 Dataset - 3 Hubs - 10 T Individual
total_running = 10
minimum_value_cab25_3_ga_sens_tind_10 = []

for i in range(total_running):
    random.seed(i)
    result = genetic_algorithm (3, 10, CAB_25_nodes_flow, CAB_25_nodes_cost, 100, 50)
    minimum_value_cab25_3_ga_sens_tind_10.append(result)

minimum_value_dataframe_cab25_3_ga_sens_tind_10 = pd.DataFrame (minimum_value_cab25_3_ga_sens_tind_10, columns=["Minimum Cost", "Allocation", "Computational Time", "Computational Time per iteration"])
minimum_value_dataframe_cab25_3_ga_sens_tind_10

,Minimum Cost,Allocation,Computational Time,Computational Time per iteration
0,8.913559e+09,"[13, 20, 20, 20, 20, 20, 13, 13, 20, 13, 13, 1...",50.650279,1.013006
1,8.397116e+09,"[4, 18, 18, 4, 4, 4, 4, 4, 4, 4, 4, 12, 4, 18,...",49.799536,0.995991
2,8.478429e+09,"[5, 17, 17, 5, 5, 5, 5, 5, 5, 5, 5, 12, 5, 5, ...",49.184613,0.983692
3,8.374475e+09,"[4, 18, 18, 4, 4, 4, 4, 4, 4, 4, 4, 12, 4, 18,...",49.995897,0.999918
4,8.411089e+09,"[2, 2, 2, 4, 4, 2, 4, 4, 4, 4, 4, 12, 4, 2, 4,...",49.990800,0.999816
5,8.397116e+09,"[4, 18, 18, 4, 4, 4, 4, 4, 4, 4, 4, 12, 4, 18,...",50.932749,1.018655
6,8.446904e+09,"[21, 2, 2, 21, 21, 2, 21, 21, 2, 21, 21, 12, 2...",50.190306,1.003806
7,8.374475e+09,"[4, 18, 18, 4, 4, 4, 4, 4, 4, 4, 4, 12, 4, 18,...",50.021554,1.000431
8,8.478429e+09,"[5, 17, 17, 5, 5, 5, 5, 5, 5, 5, 5, 12, 5, 5, ...",49.131294,0.982626
9,8.374475e+09,"[12, 18, 18, 4, 4, 4, 4, 4, 4, 4, 4, 12, 4, 18...",50.603760,1.012075


In [122]:
minimum_value_dataframe_cab25_3_ga_sens_tind_10.to_csv ('minimum_value_dataframe_cab25_3_ga_sens_tind_10.csv')

In [123]:
# Simulate Genetic Algorithm with CAB 25 Dataset - 3 Hubs - 40 T Individual
total_running = 10
minimum_value_cab25_3_ga_sens_tind_40 = []

for i in range(total_running):
    random.seed(i)
    result = genetic_algorithm (3, 40, CAB_25_nodes_flow, CAB_25_nodes_cost, 100, 50)
    minimum_value_cab25_3_ga_sens_tind_40.append(result)

minimum_value_dataframe_cab25_3_ga_sens_tind_40 = pd.DataFrame (minimum_value_cab25_3_ga_sens_tind_40, columns=["Minimum Cost", "Allocation", "Computational Time", "Computational Time per iteration"])
minimum_value_dataframe_cab25_3_ga_sens_tind_40

,Minimum Cost,Allocation,Computational Time,Computational Time per iteration
0,8.446904e+09,"[21, 2, 2, 21, 21, 2, 21, 21, 2, 21, 21, 12, 2...",50.158175,1.003163
1,8.374475e+09,"[4, 18, 18, 4, 4, 4, 4, 4, 4, 4, 4, 12, 4, 18,...",49.868206,0.997364
2,8.478429e+09,"[5, 17, 17, 5, 5, 5, 5, 5, 5, 5, 5, 12, 5, 5, ...",48.898049,0.977961
3,8.465824e+09,"[2, 2, 2, 21, 21, 2, 21, 21, 2, 21, 21, 12, 21...",50.050333,1.001007
4,8.411089e+09,"[2, 2, 2, 4, 4, 2, 4, 4, 4, 4, 4, 12, 4, 12, 4...",50.045766,1.000915
5,8.397116e+09,"[4, 18, 18, 4, 4, 4, 4, 4, 4, 4, 4, 12, 4, 18,...",49.421605,0.988432
6,8.446904e+09,"[21, 2, 2, 21, 21, 2, 21, 21, 2, 21, 21, 12, 2...",50.204898,1.004098
7,8.374475e+09,"[4, 18, 18, 4, 4, 4, 4, 4, 4, 4, 4, 12, 4, 18,...",50.980262,1.019605
8,8.478429e+09,"[5, 17, 17, 5, 5, 5, 5, 5, 5, 5, 5, 12, 5, 5, ...",48.760015,0.975200
9,8.374475e+09,"[4, 18, 18, 4, 4, 4, 4, 4, 4, 4, 4, 12, 4, 18,...",49.459277,0.989186


In [124]:
minimum_value_dataframe_cab25_3_ga_sens_tind_40.to_csv ('minimum_value_dataframe_cab25_3_ga_sens_tind_40.csv')

In [125]:
# Simulate Genetic Algorithm with CAB 25 Dataset - 3 Hubs - 60 T Individual
total_running = 10
minimum_value_cab25_3_ga_sens_tind_60 = []

for i in range(total_running):
    random.seed(i)
    result = genetic_algorithm (3, 60, CAB_25_nodes_flow, CAB_25_nodes_cost, 100, 50)
    minimum_value_cab25_3_ga_sens_tind_60.append(result)

minimum_value_dataframe_cab25_3_ga_sens_tind_60 = pd.DataFrame (minimum_value_cab25_3_ga_sens_tind_60, columns=["Minimum Cost", "Allocation", "Computational Time", "Computational Time per iteration"])
minimum_value_dataframe_cab25_3_ga_sens_tind_60

,Minimum Cost,Allocation,Computational Time,Computational Time per iteration
0,8.411089e+09,"[2, 2, 2, 4, 4, 2, 4, 4, 4, 4, 4, 12, 4, 2, 4,...",49.707541,0.994151
1,8.374475e+09,"[4, 18, 18, 4, 4, 4, 4, 4, 4, 4, 4, 12, 4, 18,...",49.222817,0.984456
2,8.374475e+09,"[4, 18, 18, 4, 4, 4, 4, 4, 18, 4, 4, 12, 4, 18...",49.516213,0.990324
3,8.374475e+09,"[4, 18, 18, 4, 4, 4, 4, 4, 4, 4, 4, 12, 4, 18,...",49.296257,0.985925
4,8.478911e+09,"[2, 2, 2, 21, 2, 2, 21, 21, 2, 21, 21, 12, 21,...",49.637932,0.992759
5,8.397116e+09,"[4, 18, 18, 4, 4, 4, 4, 4, 4, 4, 4, 12, 4, 18,...",50.589260,1.011785
6,8.446904e+09,"[21, 2, 2, 21, 21, 2, 21, 21, 2, 21, 21, 12, 2...",50.845211,1.016904
7,8.374475e+09,"[4, 18, 18, 4, 4, 4, 4, 4, 4, 4, 4, 12, 4, 18,...",50.119044,1.002381
8,8.478429e+09,"[5, 17, 17, 5, 5, 5, 5, 5, 5, 5, 5, 12, 5, 5, ...",49.141087,0.982822
9,8.374475e+09,"[4, 12, 18, 4, 4, 4, 4, 4, 4, 4, 4, 12, 4, 18,...",49.921058,0.998421


In [126]:
minimum_value_dataframe_cab25_3_ga_sens_tind_60.to_csv ('minimum_value_dataframe_cab25_3_ga_sens_tind_60.csv')

In [127]:
# Simulate Genetic Algorithm with CAB 25 Dataset - 5 Hubs - 10 T Individual
total_running = 10
minimum_value_cab25_5_ga_sens_tind_10 = []

for i in range(total_running):
    random.seed(i)
    result = genetic_algorithm (5, 10, CAB_25_nodes_flow, CAB_25_nodes_cost, 100, 50)
    minimum_value_cab25_5_ga_sens_tind_10.append(result)

minimum_value_dataframe_cab25_5_ga_sens_tind_10 = pd.DataFrame (minimum_value_cab25_5_ga_sens_tind_10, columns=["Minimum Cost", "Allocation", "Computational Time", "Computational Time per iteration"])
minimum_value_dataframe_cab25_5_ga_sens_tind_10

,Minimum Cost,Allocation,Computational Time,Computational Time per iteration
0,7.675539e+09,"[13, 17, 17, 4, 4, 4, 13, 4, 4, 13, 4, 12, 13,...",55.785357,1.115707
1,7.678738e+09,"[1, 17, 17, 4, 4, 4, 7, 4, 4, 7, 4, 12, 1, 1, ...",56.580571,1.131611
2,7.678738e+09,"[1, 17, 17, 4, 4, 4, 7, 4, 4, 7, 4, 12, 1, 1, ...",55.835328,1.116707
3,7.707931e+09,"[13, 17, 17, 4, 4, 4, 13, 4, 4, 13, 13, 12, 13...",55.705628,1.114113
4,7.865324e+09,"[1, 17, 17, 4, 6, 6, 4, 4, 6, 4, 4, 12, 1, 1, ...",55.928998,1.118580
5,7.818630e+09,"[4, 17, 17, 4, 4, 4, 4, 8, 4, 4, 4, 12, 4, 14,...",56.359221,1.127184
6,7.773952e+09,"[1, 17, 17, 4, 4, 4, 4, 8, 4, 1, 4, 12, 1, 1, ...",55.712000,1.114240
7,7.678738e+09,"[1, 17, 17, 4, 4, 4, 7, 4, 4, 7, 4, 12, 1, 1, ...",55.471782,1.109436
8,7.745475e+09,"[13, 17, 17, 4, 4, 4, 13, 13, 4, 13, 13, 12, 1...",55.282356,1.105647
9,7.784809e+09,"[1, 17, 17, 4, 4, 4, 1, 8, 4, 1, 4, 12, 1, 1, ...",55.623610,1.112472


In [128]:
minimum_value_dataframe_cab25_5_ga_sens_tind_10.to_csv ('minimum_value_dataframe_cab25_5_ga_sens_tind_10.csv')

In [129]:
# Simulate Genetic Algorithm with CAB 25 Dataset - 5 Hubs - 40 T Individual
total_running = 10
minimum_value_cab25_5_ga_sens_tind_40 = []

for i in range(total_running):
    random.seed(i)
    result = genetic_algorithm (5, 40, CAB_25_nodes_flow, CAB_25_nodes_cost, 100, 50)
    minimum_value_cab25_5_ga_sens_tind_40.append(result)

minimum_value_dataframe_cab25_5_ga_sens_tind_40 = pd.DataFrame (minimum_value_cab25_5_ga_sens_tind_40, columns=["Minimum Cost", "Allocation", "Computational Time", "Computational Time per iteration"])
minimum_value_dataframe_cab25_5_ga_sens_tind_40

,Minimum Cost,Allocation,Computational Time,Computational Time per iteration
0,7.675216e+09,"[4, 17, 17, 4, 4, 4, 7, 7, 4, 7, 4, 12, 7, 14,...",55.409700,1.108194
1,7.678738e+09,"[1, 17, 17, 4, 4, 4, 7, 4, 4, 4, 4, 12, 1, 1, ...",55.598809,1.111976
2,7.678738e+09,"[1, 17, 17, 4, 4, 4, 7, 4, 4, 7, 4, 12, 1, 1, ...",55.350739,1.107015
3,7.729898e+09,"[13, 17, 17, 4, 4, 4, 13, 13, 4, 13, 13, 12, 1...",55.871910,1.117438
4,7.842258e+09,"[1, 17, 17, 4, 6, 6, 4, 4, 6, 1, 4, 12, 4, 1, ...",55.605925,1.112118
5,7.818630e+09,"[4, 17, 17, 4, 4, 4, 4, 8, 4, 4, 4, 12, 4, 14,...",54.775744,1.095515
6,7.806341e+09,"[1, 17, 17, 4, 4, 4, 4, 8, 4, 4, 4, 12, 4, 1, ...",55.150544,1.103011
7,7.702848e+09,"[24, 17, 17, 4, 4, 4, 7, 7, 4, 7, 4, 12, 4, 24...",55.461038,1.109221
8,7.658571e+09,"[4, 17, 17, 4, 4, 4, 7, 7, 4, 7, 4, 12, 4, 14,...",55.469520,1.109390
9,7.842146e+09,"[1, 17, 17, 4, 4, 4, 21, 21, 4, 21, 21, 12, 21...",55.403065,1.108061


In [130]:
minimum_value_dataframe_cab25_5_ga_sens_tind_40.to_csv ('minimum_value_dataframe_cab25_5_ga_sens_tind_40.csv')

In [131]:
# Simulate Genetic Algorithm with CAB 25 Dataset - 5 Hubs - 40 T Individual
total_running = 10
minimum_value_cab25_5_ga_sens_tind_60 = []

for i in range(total_running):
    random.seed(i)
    result = genetic_algorithm (5, 60, CAB_25_nodes_flow, CAB_25_nodes_cost, 100, 50)
    minimum_value_cab25_5_ga_sens_tind_60.append(result)

minimum_value_dataframe_cab25_5_ga_sens_tind_60 = pd.DataFrame (minimum_value_cab25_5_ga_sens_tind_60, columns=["Minimum Cost", "Allocation", "Computational Time", "Computational Time per iteration"])
minimum_value_dataframe_cab25_5_ga_sens_tind_60

,Minimum Cost,Allocation,Computational Time,Computational Time per iteration
0,7.675539e+09,"[13, 17, 17, 4, 4, 4, 13, 4, 4, 13, 4, 12, 13,...",55.703834,1.114077
1,7.778781e+09,"[1, 17, 17, 4, 4, 4, 7, 7, 4, 7, 7, 12, 1, 1, ...",55.538406,1.110768
2,7.658571e+09,"[4, 17, 17, 4, 4, 4, 7, 7, 4, 7, 4, 12, 4, 14,...",55.629519,1.112590
3,7.675539e+09,"[13, 17, 17, 4, 4, 4, 13, 4, 4, 13, 4, 12, 13,...",56.139875,1.122798
4,7.842258e+09,"[1, 17, 17, 4, 6, 6, 4, 4, 6, 1, 4, 12, 4, 1, ...",56.152780,1.123056
5,7.818630e+09,"[4, 17, 17, 4, 4, 4, 4, 8, 4, 4, 4, 12, 4, 14,...",54.879210,1.097584
6,7.678738e+09,"[1, 17, 17, 4, 4, 4, 7, 4, 4, 7, 4, 12, 1, 1, ...",55.649851,1.112997
7,7.678738e+09,"[1, 17, 17, 4, 4, 4, 7, 4, 4, 7, 4, 12, 1, 1, ...",55.816452,1.116329
8,7.658571e+09,"[4, 17, 17, 4, 4, 4, 7, 7, 4, 7, 4, 12, 4, 14,...",55.621845,1.112437
9,7.695627e+09,"[1, 17, 17, 4, 4, 4, 7, 7, 4, 7, 4, 12, 1, 1, ...",56.157805,1.123156


In [132]:
minimum_value_dataframe_cab25_5_ga_sens_tind_60.to_csv ('minimum_value_dataframe_cab25_5_ga_sens_tind_60.csv')